In [ ]:
import numpy as np


X = np.random.rand(1000,10)
x = []
p = []
c1 = 1
c2 = 1
def Function(x):
    return 0.5 * ((x[0])**2 + (x[0] - x[1])**2 + (x[1] - x[2])**2 + (x[2])**2) - x[0]

def analytic_grad(f, x):
    return np.array([2 * x[0] - x[1] - 1, -x[0] + 2 * x[1] - x[2], - x[1] + 2 * x[2]])

def num_grad(f, x, h = 10e-4):
    return (f(x - h * np.ones(x.shape)) + f(x + h * np.ones(x.shape))) / 2 / h


def update_hessian(old_hessian, sk, yk):
    ro = 1 / (np.dot(yk.T, sk))
    I = np.eye(old_hessian.shape[0])

    term1 = I - np.dot(ro * sk, yk.T)
    term2 = I - np.dot(ro * yk, sk.T)
    term3 = np.dot(ro * sk, sk.T)

    new_hessian = np.dot(np.dot(term1, old_hessian), term2) + term3

    return new_hessian

def zoom(f, grad, x, p, c1, c2, alpha_low, alpha_high):
    grad_f = grad(f, x)
    while True:
        alpha_j = 0.5 * (alpha_low + alpha_high)
        f_j = f(x + alpha_j * p)
        if f_j > f(x) + c1 * alpha_j * grad_f or f_j >= f(x + alpha_low * p):
            alpha_high = alpha_j
        else:
            grad_f_j = grad(x + alpha_j * p)
            if np.abs(grad_f_j) <= -c2 * grad_f:
                return alpha_j
            if grad_f_j * (alpha_high - alpha_low) >= 0:
                alpha_high = alpha_low
            alpha_low = alpha_j


def line_search(f, grad, x, p, c1 = 0.01, c2 = 0.04, alpha_max = 100):
    alpha_prev = 0
    alpha_cur = (alpha_prev + alpha_max) / 2
    grad_f = grad(f, x)
    i = 1
    while True:
        f_i = f(x + alpha_cur * p)
        if (f_i > f(x) + c1 * alpha_cur * grad_f) or (f_i >= f(x + alpha_prev * p) and i > 1):
            return zoom(f, grad, x, p, c1, c2, alpha_prev, alpha_cur)
        grad_alpha_f = grad(x + alpha_cur * p)
        if np.abs(grad_alpha_f) <= -c2 * grad_f:
            return alpha_cur
        if grad_alpha_f >= 0:
            return zoom(f, grad, x, p, c1, c2, alpha_prev, alpha_cur)
        alpha_prev = alpha_cur
        alpha_cur = (alpha_prev + alpha_max) / 2
        i += 1


def BFGS(f, x_0, maxiter = 1000, eps = 10e-4, type = "analytical"):
    count = 0
    H_k = np.eye(len(x_0))
    x_k = x_0
    fgrad = 0
    if type == "analytical":
        fgrad = analytic_grad
    elif type == "numerical":
        fgrad = num_grad

    fgrad_value =fgrad(f, x_k)

    while np.linalg.norm(fgrad_value) > eps and count < maxiter:

        p_k = -np.dot(H_k, fgrad_value)
        alpha_k = line_search(f, fgrad, x_k, p_k)
        x_next = x_k + alpha_k * p_k
        fgrad_value_next = fgrad(f, x_next)
        H_k = update_hessian(H_k, x_next - x_k, fgrad_value_next - fgrad_value)
        x_k = x_next
        fgrad_value = fgrad_value_next


        count += 1

    return x_k

x_final = BFGS(f = Function, x_0 = [999., 999., 999.])

print (x_final)
